In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, LayersControl
from shapely.geometry import Point, Polygon, LineString
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
import rasterio
import rasterio.mask
from rasterio.plot import reshape_as_image 
from rasterio.plot import reshape_as_raster
from shapely.ops import unary_union

In [3]:
# Opening JSON file
f = open('Rotterdam_data/historic_towages.json')
 
# returns JSON object as a dictionary
tow_data = json.load(f)
 
# Closing file
f.close()

In [4]:
# Create empty lists to store tugs data and their original JSON index
tugs_data = []
original_index = []

# Extract tugs data into a separate list and store their original index
for i, item in enumerate(tow_data):
    for tug in item['tugs']:
        tugs_data.append(tug)
        original_index.append(i)

# Create a DataFrame for tugs data with the original index
tugs_df = pd.DataFrame(tugs_data)
tugs_df['original_index'] = original_index  # Add original index as a new column


# Create a DataFrame for the rest of the data
vessel_data = [{'from': item['from'],
               'to': item['to'],
               'vessel': item['vessel'],
               'type': item['type'],
               'additional_data': item['additional_data']}
              for item in tow_data]

vessel_df = pd.DataFrame(vessel_data)

# Display the DataFrames
tugs_df

,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,to_haven,original_index
0,9695509,244900124,VB MARS,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]","[4.05362, 51.9375]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
1,9816658,248352000,ROTTERDAM,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]","[4.05363, 51.93748]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
2,9816359,248753000,BEAGLE,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]","[4.05365, 51.93745]",None,MISSISSIPPIH EMO MH 4,None,MISS,0
3,9489936,244697000,VB KRACHT,2022-05-31 23:29:57+00:00,2022-06-01 00:30:17+00:00,"[4.0516, 51.94783]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1
4,9474905,244791000,RT ROB,2022-05-31 23:20:43+00:00,2022-06-01 00:30:17+00:00,"[4.05166, 51.94792]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
42227,9060704,245906000,THAMESBANK,2023-05-31 22:13:18+00:00,2023-05-31 22:59:18+00:00,"[4.44552, 51.8936]","[4.41163, 51.90059]",WAALH HBR B 23,None,WAAL,None,24491
42228,9507063,245932000,VB GINGER,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]","[4.1935, 51.93023]",None,CALANDK ETT EAST,None,CKVTTI,24492
42229,9060704,245906000,THAMESBANK,2023-05-31 23:09:28+00:00,2023-05-31 23:44:56+00:00,"[4.39814, 51.87972]","[4.38662, 51.89737]",ALEXANDERH ZZ 2,None,PWAH,None,24493
42230,9120190,205233000,UNION 11,2023-05-31 23:04:17+00:00,2023-05-31 23:49:58+00:00,"[4.39814, 51.87972]","[4.37126, 51.89949]",ALEXANDERH ZZ 2,None,PWAH,None,24493


In [5]:
tugs_df = tugs_df.merge(vessel_df[['type']], left_on='original_index', right_index=True)
tugs_df

,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,to_haven,original_index,type
0,9695509,244900124,VB MARS,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]","[4.05362, 51.9375]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
1,9816658,248352000,ROTTERDAM,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]","[4.05363, 51.93748]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
2,9816359,248753000,BEAGLE,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]","[4.05365, 51.93745]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming
3,9489936,244697000,VB KRACHT,2022-05-31 23:29:57+00:00,2022-06-01 00:30:17+00:00,"[4.0516, 51.94783]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving
4,9474905,244791000,RT ROB,2022-05-31 23:20:43+00:00,2022-06-01 00:30:17+00:00,"[4.05166, 51.94792]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42227,9060704,245906000,THAMESBANK,2023-05-31 22:13:18+00:00,2023-05-31 22:59:18+00:00,"[4.44552, 51.8936]","[4.41163, 51.90059]",WAALH HBR B 23,None,WAAL,None,24491,leaving
42228,9507063,245932000,VB GINGER,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]","[4.1935, 51.93023]",None,CALANDK ETT EAST,None,CKVTTI,24492,incoming
42229,9060704,245906000,THAMESBANK,2023-05-31 23:09:28+00:00,2023-05-31 23:44:56+00:00,"[4.39814, 51.87972]","[4.38662, 51.89737]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving
42230,9120190,205233000,UNION 11,2023-05-31 23:04:17+00:00,2023-05-31 23:49:58+00:00,"[4.39814, 51.87972]","[4.37126, 51.89949]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving


In [6]:
tugs_incoming = tugs_df[tugs_df['type']=='incoming']
tugs_incoming = tugs_incoming[['from',	'to',	'from_location' ,'to_haven', 'original_index']]

# Create a Shapely Point geometry from the "from_location" coordinates
tugs_incoming['geometry'] = tugs_incoming['from_location'].apply(lambda coord: Point(coord))

# Convert the DataFrame to a GeoDataFrame
tugs_incoming_gdf = gpd.GeoDataFrame(tugs_incoming, geometry='geometry')
tugs_incoming_gdf = tugs_incoming_gdf.set_crs("EPSG:4326")

In [7]:
tugs_incoming_gdf

,from,to,from_location,to_haven,original_index,geometry
0,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]",MISS,0,POINT (4.06852 51.98272)
1,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]",MISS,0,POINT (4.06852 51.98272)
2,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]",MISS,0,POINT (4.08358 51.97713)
5,2022-05-31 23:39:58+00:00,2022-06-01 00:35:27+00:00,"[4.28209, 51.89865]",BOTL,2,POINT (4.28209 51.89865)
8,2022-05-31 23:34:58+00:00,2022-06-01 01:20:40+00:00,"[4.14163, 51.96557]",3PET,4,POINT (4.14163 51.96557)
...,...,...,...,...,...,...
42214,2023-05-31 18:03:47+00:00,2023-05-31 20:05:16+00:00,"[4.12817, 51.96332]",7PET,24481,POINT (4.12817 51.96332)
42219,2023-05-31 21:01:36+00:00,2023-05-31 21:47:57+00:00,"[4.19908, 51.92873]",CKBRUG,24485,POINT (4.19908 51.92873)
42224,2023-05-31 21:17:18+00:00,2023-05-31 22:33:43+00:00,"[4.27635, 51.90111]",3PET,24489,POINT (4.27635 51.90111)
42228,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]",CKVTTI,24492,POINT (4.15890 51.94893)


tugs_incoming = tugs_df[tugs_df['type']!='incoming']
tugs_incoming = tugs_incoming[['from',	'to',	'from_location' ,'to_haven', 'original_index']]

# Create a Shapely Point geometry from the "from_location" coordinates
tugs_incoming['geometry'] = tugs_incoming['from_location'].apply(lambda coord: Point(coord))

# Convert the DataFrame to a GeoDataFrame
tugs_incoming_gdf = gpd.GeoDataFrame(tugs_incoming, geometry='geometry')
tugs_incoming_gdf = tugs_incoming_gdf.set_crs("EPSG:4326")

In [8]:
tugs_df[['from_lat', 'from_long']] = tugs_df['from_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
tugs_df[['to_lat', 'to_long']] = tugs_df['to_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
tugs_df

/var/folders/d3/q8jsq7p56m12h2f2lx6k5wl80000gn/T/ipykernel_17500/745894245.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  tugs_df[['from_lat', 'from_long']] = tugs_df['from_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
/var/folders/d3/q8jsq7p56m12h2f2lx6k5wl80000gn/T/ipykernel_17500/745894245.py:2: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  tugs_df[['to_lat', 'to_long']] = tugs_df['to_location'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)


,imo,mmsi,name,from,to,from_location,to_location,from_berth,to_berth,from_haven,to_haven,original_index,type,from_lat,from_long,to_lat,to_long
0,9695509,244900124,VB MARS,2022-05-31 22:49:33+00:00,2022-06-01 00:10:13+00:00,"[4.06852, 51.98272]","[4.05362, 51.9375]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming,4.06852,51.98272,4.05362,51.93750
1,9816658,248352000,ROTTERDAM,2022-05-31 22:49:32+00:00,2022-06-01 00:20:21+00:00,"[4.06852, 51.98272]","[4.05363, 51.93748]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming,4.06852,51.98272,4.05363,51.93748
2,9816359,248753000,BEAGLE,2022-05-31 22:54:16+00:00,2022-06-01 00:35:37+00:00,"[4.08358, 51.97713]","[4.05365, 51.93745]",None,MISSISSIPPIH EMO MH 4,None,MISS,0,incoming,4.08358,51.97713,4.05365,51.93745
3,9489936,244697000,VB KRACHT,2022-05-31 23:29:57+00:00,2022-06-01 00:30:17+00:00,"[4.0516, 51.94783]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving,4.05160,51.94783,4.08905,51.96774
4,9474905,244791000,RT ROB,2022-05-31 23:20:43+00:00,2022-06-01 00:30:17+00:00,"[4.05166, 51.94792]","[4.08905, 51.96774]",AMAZONEH ECT DDE,None,AMAZ,None,1,leaving,4.05166,51.94792,4.08905,51.96774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42227,9060704,245906000,THAMESBANK,2023-05-31 22:13:18+00:00,2023-05-31 22:59:18+00:00,"[4.44552, 51.8936]","[4.41163, 51.90059]",WAALH HBR B 23,None,WAAL,None,24491,leaving,4.44552,51.89360,4.41163,51.90059
42228,9507063,245932000,VB GINGER,2023-05-31 23:04:20+00:00,2023-05-31 23:44:50+00:00,"[4.1589, 51.94893]","[4.1935, 51.93023]",None,CALANDK ETT EAST,None,CKVTTI,24492,incoming,4.15890,51.94893,4.19350,51.93023
42229,9060704,245906000,THAMESBANK,2023-05-31 23:09:28+00:00,2023-05-31 23:44:56+00:00,"[4.39814, 51.87972]","[4.38662, 51.89737]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving,4.39814,51.87972,4.38662,51.89737
42230,9120190,205233000,UNION 11,2023-05-31 23:04:17+00:00,2023-05-31 23:49:58+00:00,"[4.39814, 51.87972]","[4.37126, 51.89949]",ALEXANDERH ZZ 2,None,PWAH,None,24493,leaving,4.39814,51.87972,4.37126,51.89949


Import other infos

In [9]:
def flatten_dict(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

# Opening JSON file
f = open('Rotterdam_data/ais_rotterdam/ais_rotterdam_1.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
# Flatten each dictionary and store the results in a list
flattened_dicts = [flatten_dict(d) for d in data['data']]

# Create a DataFrame
data = pd.DataFrame(flattened_dicts)

# Closing file
f.close()

In [1]:
data

NameError: name 'data' is not defined

In [13]:
data['navigation_destination_eta'] = pd.to_datetime(data['navigation_destination_eta'])
data['navigation_time'] = pd.to_datetime(data['navigation_time'])
#data = data.drop_duplicates()
data[['navigation_location_lat', 'navigation_location_long']] = data['navigation_location_coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(','))).astype(float)
data

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

For the "from"

In [ ]:
# Define categorical and numeric columns
#categorical_cols = ['vessel_type', 'vessel_callsign', 'vessel_subtype', 'vessel_name', 'navigation_status']
features = ['vessel_imo', 'navigation_draught', 'navigation_location_long', 'navigation_location_lat','navigation_speed', 'navigation_course', 'device_mmsi', 'device_dimensions_to_bow','device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port', 'navigation_draught','navigation_heading','navigation_course', 'navigation_speed']

""" # Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Split the data into training and testing sets
X = datad.drop(columns=['meeting']) """

#features = data[['vessel_imo', 'navigation_draught', 'navigation_location_long', 'navigation_location_lat','navigation_speed', 'navigation_course', 'device_mmsi', 'device_dimensions_to_bow','device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port']]
X = features.values

# Assuming 'latitude' and 'longitude' are your target variables (y-values)
# Target variables 
y_from_lat = tugs_df['from_lat'].values
y_from_lon = tugs_df['from_long'].values


Linear Regression

In [ ]:
# Split data into training and testing sets for latitude prediction
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(X, y_from_lat, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for latitude
linear_model_lat = LinearRegression()
linear_model_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = linear_model_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Linear Regression Mean Squared Error (Latitude): {mse_lat}")

# Split data into training and testing sets for longitude prediction
X_from_lon_train, X_from_lon_test, y_from_lon_train, y_from_lon_test = train_test_split(X, y_from_lon, test_size=0.2, random_state=42)

# Initialize and train the linear regression model for longitude
linear_model_lon = LinearRegression()
linear_model_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = linear_model_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Linear Regression Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
map_center = [mean_latitude, mean_longitude]
a = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(3):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(a)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(a)

# Save the map as an HTML files
a.save('map_linear_regression.html')



Decision Tree

In [ ]:
# Initialize and train the decision tree regression model for latitude
decision_tree_lat = DecisionTreeRegressor(random_state=42)
decision_tree_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = decision_tree_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Decision Tree Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the decision tree regression model for longitude
decision_tree_lon = DecisionTreeRegressor(random_state=42)
decision_tree_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = decision_tree_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Decision Tree Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
map_center = [mean_latitude, mean_longitude]
b = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(3):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(b)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(b)

# Save the map as an HTML file
b.save('map_decision_tree.html')



Random Forest Regression Model

In [ ]:
# Initialize and train the random forest regression model for latitude
random_forest_lat = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = random_forest_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Random Forest Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the random forest regression model for longitude
random_forest_lon = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = random_forest_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Random Forest Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
map_center = [mean_latitude, mean_longitude]
c = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(3):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(c)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(c)

# Save the map as an HTML file
c.save('map_random_forest.html')




Gradient Boosting Regression Model

In [ ]:
# Initialize and train the gradient boosting regression model for latitude
gradient_boosting_lat = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lat.fit(X_from_lat_train, y_from_lat_train)

# Make predictions for latitude
predictions_lat = gradient_boosting_lat.predict(X_from_lat_test)

# Calculate mean squared error for latitude prediction
mse_lat = mean_squared_error(y_from_lat_test, predictions_lat)
print(f"Gradient Boosting Mean Squared Error (Latitude): {mse_lat}")

# Initialize and train the gradient boosting regression model for longitude
gradient_boosting_lon = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_lon.fit(X_from_lon_train, y_from_lon_train)

# Make predictions for longitude
predictions_lon = gradient_boosting_lon.predict(X_from_lon_test)

# Calculate mean squared error for longitude prediction
mse_lon = mean_squared_error(y_from_lon_test, predictions_lon)
print(f"Gradient Boosting Mean Squared Error (Longitude): {mse_lon}")

# Calculate geodesic distance between true and predicted coordinates in meters
for i in range(len(predictions_lat)):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    distance = geodesic(true_coords, pred_coords).meters
    print(f"Geodesic Distance (meter) for Test Point {i+1}: {distance}")

# Create a map centered at a specific location (e.g., mean latitude and longitude of your data)
map_center = [mean_latitude, mean_longitude]
d = folium.Map(location=map_center, zoom_start=12)

# Add real and predicted locations to the map for the first 3 results
for i in range(3):
    true_coords = (y_from_lat_test[i], y_from_lon_test[i])
    pred_coords = (predictions_lat[i], predictions_lon[i])
    folium.Marker(location=true_coords, popup=f'Real: {true_coords}', icon=folium.Icon(color='green')).add_to(d)
    folium.Marker(location=pred_coords, popup=f'Predicted: {pred_coords}', icon=folium.Icon(color='blue')).add_to(decision_tree_lat)

# Save the map as an HTML file
d.save('map_gradient_boosting.html')

